In [2]:
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import translators as ts
import undetected_chromedriver as uc
import pandas as pd
from selenium import webdriver
import time
from pymongo import MongoClient
from datetime import datetime
import csv
from google_trans_new import google_translator
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0 
from collections import Counter
from bson.objectid import ObjectId
import requests

In [3]:
ts_apis = ['bing', 'google', 'baidu', 'yandex', 'alibaba', 'apertium', 
           'argos', 'caiyun', 'cloudTranslation', 'deepl', 'elia', 
           'hujiang', 'iciba', 'iflytek', 'iflyrec', 'itranslate', 
           'judic', 'languageWire', 'lingvanex', 'mglip', 'mirai', 
           'modernMt', 'myMemory', 'niutrans', 'papago', 'qqFanyi', 
           'qqTranSmart', 'reverso', 'sogou', 'sysTran', 'tilde', 
           'translateCom', 'translateMe', 'utibet', 'volcEngine',
           'yeekit', 'youdao',]

In [4]:
client = MongoClient('mongodb://192.168.1.11:27017/')

# Selecciona la base de datos
db = client['recopilarnovelas']
coleccion_app_novela = db['app_novela']
coleccion_app_capitulo = db['app_capitulo']
coleccion_app_contenidocapitulo = db['app_contenidocapitulo']

In [5]:
def traducir(texto):
    contenido_p = ''
    for api in ts_apis:
        try:
            contenido_p = ts.translate_text(
            texto, translator=api, to_language='es')
            break
        except Exception as e:
            contenido_p = traducir(texto)
            break
    return contenido_p

In [6]:
# try:
#     df_list_novels = pd.read_csv('chapters/boxnovel_novelas.csv', sep=';', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
# except:
#     df_list_novels = pd.DataFrame(columns=['nombre', 'url'])
# df_list_novels
# df_list_novels = pd.DataFrame(columns=['nombre', 'url'])
# df_existing_novels =pd.read_csv('existing_novels.csv', encoding='utf-8-sig', sep=';', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
# df_existing_novels
df_list_novels = pd.DataFrame(columns=['nombre', 'url'])

In [7]:
# options = webdriver.ChromeOptions()
options = webdriver.FirefoxOptions()
options.add_argument('--no-daemon')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')

In [8]:
url = 'https://boxnovel.com/novel/'

In [9]:
# driver = uc.Chrome(options=options, service = Service(executable_path=ChromeDriverManager().install()))
# driver = webdriver.Firefox(options=options, service=FirefoxService(GeckoDriverManager().install()))
driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))

In [10]:
driver.get(url)
time.sleep(5)
# driver.maximize_window()

In [ ]:
By.__dict__

In [12]:
if df_list_novels.empty:
    list_novels = {"nombre":[],"url":[]}
    idx = 2
    while idx <=163:
        try:
            html = driver.page_source
            soup = bs(html, 'html.parser')
            page_listing_item = soup.find_all('div', class_='page-listing-item')
            # print(page_listing_item)
            for page_list in page_listing_item:
                for item in page_list.find_all('div', class_='item-summary'):
                    title = item.find('div', class_='post-title').find('a')
                    title_text = ''.join([x for x in title.getText() if x.isalnum() or x == ' '])
                    title_url = title.get('href')
                    print(f"{title_text} {title_url}")
                    list_novels['nombre'].append(title_text)
                    list_novels['url'].append(title_url)
            driver.get(f"{url}/page/{idx}")
            # time.sleep(1)
        except:
            pass
        idx+=1

vsd https://bronovel.com/novels/vsd/
vsdv https://bronovel.com/novels/vsdv/
Swallowed Star https://bronovel.com/novels/swallowed-star/
1[{"variableName": "ID_TO_MEANING", "type": "dictionary", "supportedEngines": ["pandas"], "isLocalVariable": true, "rawType": "builtins.dict"}, {"variableName": "NULL", "type": "unknown", "supportedEngines": ["pandas"], "isLocalVariable": true, "rawType": "_pydevd_bundle.pydevd_constants.Null"}]



In [ ]:
df_list_novels = pd.DataFrame(list_novels)
df_list_novels.to_csv(r'D:\PycharmProjects\novelasligeraswebscrapping\chapters\boxnovel_novelas.csv', index=False, encoding='utf-8', sep=';', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
df_list_novels

In [ ]:
def get_novel_data(driver, novel_url):
    # driver.get(novel_url)
    # time.sleep(5)
    html_novel = driver.page_source
    soup_novel = bs(html_novel, 'html.parser')
    title = soup_novel.find('div', class_='post-title').find('h1').getText().strip().rstrip()
    title = ''.join([x for x in title if x.isalnum() or x == ' '])
    tab_summary = soup_novel.find('div',class_='tab-summary')
    post_content = tab_summary.find('div', class_='post-content').find_all('div', class_='post-content_item')
    post_status = tab_summary.find('div', class_='post-status').find_all('div', class_='post-content_item')
    author = ''
    genre = ''
    status = ''
    for div_item in post_content:
        cabecera_resumen = div_item.find('div', class_='summary-heading').find('h5').getText().strip().rstrip()
        
        if 'author' in cabecera_resumen.lower():
            author = div_item.find('div', class_='summary-content').getText().strip().rstrip()
        if 'genre' in cabecera_resumen.lower():
            genre = div_item.find('div', class_='summary-content').getText().strip().rstrip()
    for div_item in post_status:
        cabecera_resumen = div_item.find('div', class_='summary-heading').find('h5').getText().strip().rstrip()
        if 'status' in cabecera_resumen.lower():
            status = div_item.find('div', class_='summary-content').getText().strip().rstrip()

    try:
        img_src = tab_summary.find('div', class_='summary_image').find('img').get('src')
    except:
        img_src = ''

    try:
        description = soup_novel.find('div', class_='description-summary').find('div', id='editdescription').getText().strip().rstrip()
    except:
        try:
            description = soup_novel.find('div', class_='description-summary').find('div', class_='summary__content').getText().strip().rstrip()
        except:
            description = 'No Description.'

    novel_data = {
        "sitio_id": "66b2b6f538c8afe8512e31aa",
        "nombre": title.upper(),
        "sinopsis": description,
        "autor": author,
        "genero": genre,
        "status": 'emision' if status == 'Ongoing' else 'completo',
        "url": novel_url,
        "imagen_url": img_src,
        "created_at": datetime.now(),
        "updated_at": datetime.now()
    }
    return novel_data

In [ ]:
def get_chapter_data(driver, url_novel, novel_id, collection):
    # driver.get(url_novel)
    # time.sleep(5)
    html_novel = driver.page_source
    soup_novel = bs(html_novel, 'html.parser')
    panel_chapter = soup_novel.find('div', class_='page-content-listing').find('ul', class_='main').find_all('li')
    panel_chapter = panel_chapter[::-1]

    existing_chapters = {str(x['nombre']).strip().rstrip():str(x['_id']) for x in collection.find({'novela_id': novel_id}, {'_id':1,'nombre': 1})}

    for row in panel_chapter:
        nombre_cap = row.find('a').getText().strip().rstrip()
        chapter_url = row.find('a').get('href')
        if nombre_cap not in existing_chapters.keys():
            chapter_data = {
                "novela_id": novel_id,
                "nombre": nombre_cap,
                "url": chapter_url,
                "created_at": datetime.now(),
                "updated_at": datetime.now()
            }
            collection.insert_one(chapter_data)
            print(f'Url de {nombre_cap} ha sido insertada')
        else:
            resultado = collection.update_one(
                {'_id': ObjectId(existing_chapters[nombre_cap])},
                {'$set': {'url': chapter_url}}
            )
            print(f'Url de {nombre_cap} actualizada: {resultado.modified_count}')

In [ ]:
# existing_novels =  [{'_id': str(x['_id']),'nombre': x['nombre'], 'url': x['url']} for x in coleccion_app_novela.find({}, {'_id': 1, 'nombre':1, 'url': 1})]
# print(existing_novels[:5])

In [ ]:
existing_novels =  [{'_id': str(x['_id']),'nombre': x['nombre'], 'url': x['url']} for x in coleccion_app_novela.find({'sitio_id':'66b2b6f538c8afe8512e31aa'}, {'_id': 1, 'nombre':1, 'url': 1})]
existing_novels_names = {x['nombre']:x['_id'] for x in existing_novels}
existing_novels_url = {x['url']:x['_id'] for x in existing_novels}
for idx, novel in df_list_novels.iterrows():
    # driver.get(novel['url'])
    if str(novel['url']).strip().rstrip() in existing_novels_url.keys():
        novel_id = existing_novels_url[str(novel['url']).strip().rstrip()]
        print(f"{idx} {novel_id} {str(novel['nombre']).upper().strip().rstrip()} {novel['url']} ya existe, hay que actualizar la URL")
        pass
        driver.get(novel['url'])
        time.sleep(5)
        resultado = coleccion_app_novela.update_one(
            {'_id': ObjectId(novel_id)},
            {'$set': {'url': novel['url']}}
        )
        print(f'Documentos actualizados: {resultado.modified_count}')
        try:
            get_chapter_data(driver, novel['url'], novel_id, coleccion_app_capitulo)
            print(f"Todos los capítulos publicados de '{str(novel['nombre']).upper().strip().rstrip()}' han sido ingresados a la Base de Datos.")
        except Exception as e:
            print('=======================================================================================================================')
            print(f"Error al procesar la novela '{novel['url']}': {e}")
            print('=======================================================================================================================')
            pass
    else:
        print(f"{idx} {str(novel['nombre']).upper().strip().rstrip()} {novel['url']} no existe, hay que crear la novela")
        driver.get(novel['url'])
        time.sleep(5)
        novel_data = get_novel_data(driver, novel['url'])
        novel_id = str(coleccion_app_novela.insert_one(novel_data).inserted_id)
        print(f"Novela '{novel_data['nombre']}' ingresada a la Base de Datos.")
        try:
            get_chapter_data(driver, novel['url'], novel_id, coleccion_app_capitulo)
            print(f"Todos los capítulos publicados de '{str(novel['nombre']).upper().strip().rstrip()}' han sido ingresados a la Base de Datos.")
        except Exception as e:
            print('=======================================================================================================================')
            print(f"Error al procesar la novela {idx} ==> {novel['nombre']}: {e}")
            print('=======================================================================================================================')
            pass
    # break

In [ ]:
# coleccion_app_capitulo.delete_many({'novela_id': '666f9673d39ea517840211d4'})

In [ ]:
# existing_novels =  [{'_id': str(x['_id']),'nombre': x['nombre'], 'url': x['url']} for x in coleccion_app_novela.find({'sitio_id':'65e7e19f7963e189aa4c1258'}, {'_id': 1, 'nombre':1, 'url': 1})]
# for novel in existing_novels:
#     print(novel['_id'], end=' | ')
#     coleccion_app_capitulo.delete_many({'novela_id': str(novel['_id'])})
# coleccion_app_novela.delete_many({'sitio_id':'65e7e19f7963e189aa4c1258'})

In [ ]:
existing_novels =  [{'_id': str(x['_id']),'nombre': x['nombre'], 'url': x['url']} for x in coleccion_app_novela.find({'sitio_id':'66b2b6f538c8afe8512e31aa'}, {'_id': 1, 'nombre':1, 'url': 1})]
df_existing_novels = pd.DataFrame(existing_novels)
df_existing_novels

In [ ]:
df_existing_novels.to_csv(r'D:\PycharmProjects\novelasligeraswebscrapping\chapters\existing_novels_boxnovel.csv', index=False, encoding='utf-8-sig', sep=';', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
def enviar_contenido_capitulo(novel_id,capitulo_id,texto_capitulo):
    url_contenido_capitulos = "http://192.168.1.11:8000/api/contenidocapitulos/"
    headers = {
        'Content-Type': 'application/json'
    }
    payload_chapter = {
        "novela_id": novel_id,
        "capitulo_id": capitulo_id,
        "texto": texto_capitulo,
        "created_at": datetime.now().isoformat(),
        "updated_at": datetime.now().isoformat()
    }

    try:
        res_cap = requests.post(url_contenido_capitulos, headers=headers, json=payload_chapter)
        res_cap.raise_for_status()
        return res_cap.json()['_id']
    except requests.exceptions.RequestException as e:
        print(f"Error al enviar el contenido del capítulo: {e}")
        return None

In [ ]:
for idx, row in df_existing_novels.iterrows():
    # if row['_id'] == '66b2cdb625b0afb7ede0022c':
    capitulos_novela = pd.DataFrame([{'_id': str(x['_id']),'nombre': x['nombre'], 'url': x['url']} for x in coleccion_app_capitulo.find({'novela_id':row['_id']}, {'_id': 1, 'nombre':1, 'url': 1}).sort('created_at', 1)])
    contenidocapitulos_novela = set([str(x['capitulo_id']) for x in coleccion_app_contenidocapitulo.find({'novela_id':row['_id']}, {'_id': 1, 'capitulo_id':1})])
    # driver.get(row['url'])
    print(f"Novela {row['nombre']} id: {row['_id']}")
    for idx2, cap in capitulos_novela.iterrows():
        if cap['_id'] not in contenidocapitulos_novela:
            driver.get(cap['url'])
            html_novel_cap = driver.page_source
            soup_novel_cap = bs(html_novel_cap, 'html.parser')
            content = soup_novel_cap.find('div', class_='reading-content')
            content_p = content.find_all('p')
            texto_capitulo = ''
            for p in content_p:
                if len(p.getText()) > 4000:
                    texto_traducido = '. '.join([traducir(x) for x in p.getText().strip().rstrip().split('. ') if x != ''])
                    texto_capitulo += f"<p>{texto_traducido}</p>"
                else:
                    texto_traducido = traducir(p.getText())
                    texto_capitulo += f"<p>{texto_traducido}</p>"
                
            _id = enviar_contenido_capitulo(novel_id=row['_id'], capitulo_id=cap['_id'], texto_capitulo=texto_capitulo)
            # print(f"Capitulo {cap['nombre']} insertado con la ID: {_id}")
            # print(content_p)
        # else:
        #     print(f"Capitulo {cap['nombre']} ya se encuentra en la base de datos con la ID: {cap['_id']}")
    # break


In [ ]:
driver.quit()